# To get data and labels:
```
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
tar -xvf images.tar.gz
tar -xvf annotations.tar.gz
```

# Convert to TFRecord format
```
python ../models/research/object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path=../models/research/object_detection/data/pet_label_map.pbtxt \
    --data_dir=pet_detector \
    --output_dir=pet_detector
```

# Download pretrained model
```
wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
tar -xvf faster_rcnn_resnet50_coco_2018_01_28.tar.gz
```

# Object detection pipeline
```
cp pet_detector/faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.* pet_detector/
cp ../models/research/object_detection/data/pet_label_map.pbtxt pet_detector/
cp ../models/research/object_detection/samples/configs/faster_rcnn_resnet50_pets.config pet_detector/
sed -i 's|PATH_TO_BE_CONFIGURED|/home/rhermoza/github/obj-detection/pet_detector|g' pet_detector/faster_rcnn_resnet50_coco_2018_01_28/faster_rcnn_resnet50_pets.config
```

# Create train and eval folders
```
mkdir pet_detector/train
mkdir pet_detector/eval
```

# Install pycocotools
```
git clone https://github.com/pdollar/coco.git

cd coco/PythonAPI
make
make install
python setup.py install
```

# Run train job
```
python ../models/research/object_detection/train.py \
    --logtostderr \
    --pipeline_config_path=pet_detector/faster_rcnn_resnet50_pets.config \
    --train_dir=pet_detector/train
```

# Run eval job
```
python ../models/research/object_detection/eval_cpu.py \
    --logtostderr \
    --pipeline_config_path=pet_detector/faster_rcnn_resnet50_pets.config \
    --checkpoint_dir=pet_detector/train \
    --eval_dir=pet_detector/eval
```
eval_cpu.py have this line added to force no gpu use:
```
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
```

#  Start tensorboard
```
tensorboard --logdir=pet_detector
```
# Exporting the Tensorflow Graph
```
# note the checkpoint number (ex: 44594)
python ../models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path pet_detector/faster_rcnn_resnet50_pets.config \
    --trained_checkpoint_prefix pet_detector/train/model.ckpt-44594 \
    --output_directory pet_detector/exported_graphs
```


In [72]:
!ls pet_detector/exported_graphs/

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config
